<a href="https://colab.research.google.com/github/LorenzoBellomo/InformationRetrieval/blob/main/notebooks/5_GraphDBs_short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphDBs and Neo4j

In this lecture, we focus on GraphDBs, using Neo4j and its language (Cypher) to make some simple queries.

In [ ]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install yfiles_jupyter_graphs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
from neo4j import GraphDatabase

Go to the neo4j sandbox and click on the box "blank project" ([https://sandbox.neo4j.com/](https://sandbox.neo4j.com/)). After the sandbox has been created, click on the rightmost black arrow, click on "connections details", and copy the BOLT URL and password in the command just below

In [ ]:
driver = GraphDatabase.driver("<INSERT BOLT URL>", auth=("neo4j", "<INSERT PASSWORD>"))
session = driver.session(database="neo4j")
driver.execute_query("MATCH (n) DETACH DELETE n  ") # purging eventual graphs already loaded

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7d1d025867d0>, keys=[])

Now we are ready to populate the graph

In [ ]:
driver.execute_query(
    "CREATE (john:Person {name: \"John\"})   \
    CREATE (joe:Person {name: \"Joe\"})      \
    CREATE (steve:Person {name: \"Steve\"})  \
    CREATE (sara:Person {name: \"Sara\"})    \
    CREATE (maria:Person {name: \"Maria\"})  \
    CREATE (john)-[:FRIEND]->(joe)-[:FRIEND]->(steve)    \
    CREATE (john)-[:FRIEND]->(sara)-[:FRIEND]->(maria)   \
")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7d1d0259b090>, keys=[])

In [ ]:
records, summary, keys = driver.execute_query("MATCH (n) return n") # n is reserved to access nodes

Let's print the results, results are a list of "node entries". To access them we need to use the "name" of the variable that we used in the query (n)

In [ ]:
for record in records:
    # iterating over all nodes
    elem = record["n"]
    print("{} node with name {}".format(list(elem.labels)[0], elem["name"])) # neo4j supports multiple labels (types) per node and edge

Person node with name Maria
Person node with name John
Person node with name Joe
Person node with name Steve
Person node with name Sara


In [ ]:
print("KEYS: ", keys)
# Here is the documentationo of all that you can find on the Summary object (https://neo4j.com/docs/python-manual/5/result-summary/)

KEYS:  ['n']
SUMMARY (counters):  {}
SUMMARY (args):  None


Let's use this widget library to see the graph visually (for ease of understanding)

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from google.colab import output
output.enable_custom_widget_manager()
def draw_graph():
    g = session.run("MATCH (n) -[e]-> (m) return n, m, e").graph()
    gw = GraphWidget(graph=g)
    return gw

draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Now we try to add a new edge from nodes "Sara" and "Joe"

In [ ]:
results = driver.execute_query("CREATE (j:Person {name: \"Sara\"})-[rel:FRIEND]->(m:Person {name: \"Joe\"})  ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Oops, we meessed up, we did not "bind" the two names to the nodes already present. First, let's delete the two new nodes we created.

In [ ]:
results = (driver.execute_query("MATCH (n {name: \"Sara\"}) -[]-> (m {name: \"Joe\"}) RETURN n, m"))
res = results[0][0]
print(res)
print(res["n"].element_id)
print(res["m"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["n"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["m"].element_id)
draw_graph()

<Record n=<Node element_id='4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:5' labels=frozenset({'Person'}) properties={'name': 'Sara'}> m=<Node element_id='4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:6' labels=frozenset({'Person'}) properties={'name': 'Joe'}>>
4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:5
4:1584a58c-6298-4af9-ac3f-2dab48b8bfb1:6


GraphWidget(layout=Layout(height='500px', width='100%'))

To actually add the edge we wanted to add, we need to use the "MERGE" statement

In [ ]:
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (m:Person {name: \"Joe\"})  MERGE (j)-[r:FRIEND]->(m)")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Let's now add two countries, and let's connect our nodes to those countries

In [ ]:
driver.execute_query("CREATE (:Country {name: \"Australia\"})  CREATE (:Country {name: \"Germany\"}) ")
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"Maria\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"John\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"Joe\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
driver.execute_query("MATCH (j:Person {name: \"Steve\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l) ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

Testing regex in the query

In [ ]:
results = driver.execute_query("MATCH (p:Person) WHERE p.name =~ 'Jo.*' RETURN p.name  ")
for res in results[0]:
    print(res["p.name"])

John
Joe


Testing the "exists" keyword

In [ ]:
results = driver.execute_query("MATCH (p:Person)-[r:FRIEND]->(m:Person)  WHERE exists((m)-[:FRIEND]->(p))  RETURN m, p ")
for res in results[0]:
    print(res["m"]["name"], " -> ", res["p"]["name"])

aggregate and count

In [ ]:
# we use the keyword "as" to name the adjacency list (retrieved with collect) and its size (retrieved with count)
results = driver.execute_query("MATCH (p1:Person)-[:FRIEND]->(p2:Person) RETURN p1.name, collect(p2.name) as listOfFriends, count(*) as numberOfFriends")
for res in results[0]:
  print("{} has {} friends {}".format(res["p1.name"], res["numberOfFriends"], res["listOfFriends"]))

John has 2 friends ['Joe', 'Sara']
Joe has 1 friends ['Steve']
Sara has 2 friends ['Joe', 'Maria']


Let's now check how directed edges work. This next query uses unidirect edges (-[related_to]-).

Try changing it to directed ones (-[related_to]->).

In [ ]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[y]-(x)  return x.name, type(y), Labels(x)[0]")
for res in results[0]:
  print("Maria -[{}]-> {} ({})".format(res["type(y)"], res["x.name"], res["Labels(x)[0]"]))

Maria -[FRIEND]-> Sara (Person)
Maria -[LIVES_IN]-> Germany (Country)


indirect paths of max length 2 from Maria

In [ ]:
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]-(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"]["name"])

Sara
Germany
Joe
John


direct paths of max length 2 from Maria

In [ ]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]->(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"]["name"])

Germany


Check the database schema, to see which kind of edges connect which kinds of nodes

In [ ]:
g = session.run("CALL db.schema.visualization()").graph()
GraphWidget(graph=g)

GraphWidget(layout=Layout(height='500px', width='100%'))

To run large scale data science algorithms, we need to create an in-memory projection of our graph

In [ ]:
# "in_memory_graph" is the name we assign to the RAM projection of our graph
results = driver.execute_query("CALL gds.graph.project(\'in_memory_graph\', \
    [\'Person\', \'Country\'], [\'FRIEND\', \'LIVES_IN\'])  \
    YIELD graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipCount AS rels  ")
print(results[0])

[<Record graph='in_memory_graph' nodeProjection={'Country': {'label': 'Country', 'properties': {}}, 'Person': {'label': 'Person', 'properties': {}}} nodes=7 rels=10>]


let's compute the page rank of all nodes. Higher pagerank nodes are "more important" ones.

In [ ]:
results = driver.execute_query("""CALL gds.pageRank.stream(\'in_memory_graph\')
    YIELD nodeId, score WITH gds.util.asNode(nodeId) AS n,score AS pageRank
    RETURN n.name AS name, Labels(n)[0] AS type, pageRank ORDER BY pageRank DESC """)
for res in results[0]:
  # sorted by decreasing page rank
  print(res["name"], " - ", res["type"], " - ", round(res["pageRank"]*100)/100)

Australia  -  Country  -  0.47
Germany  -  Country  -  0.42
Steve  -  Person  -  0.25
Joe  -  Person  -  0.25
Maria  -  Person  -  0.2
Sara  -  Person  -  0.19
John  -  Person  -  0.15


Let's also compute the shortest path between John and Maria

In [ ]:
results = driver.execute_query("""MATCH (source:Person {name: \'John\'}) MATCH (target:Person {name: \'Maria\'})
    CALL gds.shortestPath.dijkstra.stream(\'in_memory_graph\', {    sourceNode: source,
    targetNode: target}) YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN    index, totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs ORDER BY index""")

for res in results[0]:
    print("PATH is {} (its incremental cost is {}, for a total cost of {})".format(res["nodeNames"], res["costs"], res["totalCost"]))

PATH is ['John', 'Sara', 'Maria'] (its incremental cost is [0.0, 1.0, 2.0], for a total cost of 2.0)
